# API Revision de XML e integracion en Base de datos en la nube

In [367]:
import os
import pandas as pd
import pymongo
import xmltodict
from pandas.io.json import json_normalize

import warnings
warnings.filterwarnings("ignore")

import io
from os import listdir, remove
from os.path import isfile, join
import pathlib
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore")


In [368]:
filespath='./temp/'
fileslist = [f for f in listdir(filespath) if isfile(join(filespath, f))]
for i in fileslist:
    if i[-3:]!="xml":
        fileslist.remove(i)


In [401]:
for k,z in tqdm(enumerate(fileslist)): 
    #Opening File
    xml=open('./temp/'+z,'r')#,encoding=predict_encoding('./temp/'+z))
    xml_parsed=xmltodict.parse(xml.read())
    
    #Extending XML
    xml_keys=list(xml_parsed['cfdi:Comprobante'].keys())
    xml_values=list(xml_parsed['cfdi:Comprobante'].values())
    xml_df=pd.DataFrame(xml_values).T
    xml_df.columns=xml_keys

    #Descartando columnas no útiles (por el momento)
    columnas=['@xmlns:cfdi','@xmlns:xsi','@Certificado','@xsi:schemaLocation','@Sello','cfdi:Complemento']
    for i in columnas:
        try:
            xml_df.drop([i],axis=1,inplace=True)
        except:
            pass
    
    #Elementos para hacer sub despliegue
    list_sub=['cfdi:Emisor','cfdi:Receptor'] #cfdi:Conceptos tiene una estructura  de mas grandos de profundidad

    #Sub despliegue de elementos en list_sub
    for i in list_sub:
        #print(i)
        for j in list(dict(xml_df[i][0]).keys()):
            #print(i+'_'+j)
            xml_df[i+'_'+j]=xml_df[i][0][j]

    #Quitar columnas con diccionarios
    xml_df.drop(['cfdi:Emisor','cfdi:Receptor'],axis=1,inplace=True)
    

    #Limpiar nombres columna
    for i,e in enumerate(xml_df.columns):
        xml_df.rename(columns={str(e):str(e).replace('cfdi:','')},inplace=True)
    for i,e in enumerate(xml_df.columns):
        xml_df.rename(columns={str(e):str(e).replace('@','')},inplace=True)
    for i,e in enumerate(xml_df.columns):
        xml_df.rename(columns={str(e):str(e).lower()},inplace=True)
    

    #Optimizar tipos de datos
    xml_df.fecha[0] = datetime.strptime(xml_df.fecha[0], '%Y-%m-%dT%H:%M:%S')
    xml_df.fecha

    for i in xml_df.columns:    
        try:
            xml_df[i] = pd.to_numeric(xml_df[i])

            if type(xml_df[i])==float:
                xml_df[i] = pd.to_numeric(xml_df[i], downcast='float')
            elif type(xml_df[i])==int:
                xml_df[i] = pd.to_numeric(xml_df[i], downcast='int')
        except:
            pass

    #Ingresar xml completo como respaldo
    xml_df['xml']=xml
    
    #Cerrar archivo
    xml.close()
    
    #Integracion de la base de datos final
    if k == 0: 
        df_xml = xml_df #Crea base de datos de salida en la primera iteracion
    else:
        columnas_nuevas = list(xml_df.columns)
        columnas_existentes = list(df_xml.columns)
        
        #Busca columnas nuevas en el nuevo registro y crea la columna correspondiente
        for e in columnas_existentes:
            try:
                columnas_nuevas=columnas_nuevas.remove(e)
            except:
                pass
        
        try:
            for e in columnas_nuevas:
                df_xml[e] = df_xml[e].apply(lambda x: None)
        except:
            pass
        
        
        #copia la columna anterior con las nuevas columnas
        df_xml=df_xml.append(df_xml.iloc[0],ignore_index=True)
        
        #Ingreso de los registros uno a uno en donde pueden no existir todos los campos en el nuevo registro 
        columnas=df_xml.columns
        
        #print(xml_df['emisor_rfc'][0])
        #print(z)
        
        for i in columnas:
            try:
                df_xml[i].iloc[-1]=xml_df[i][0]
            except:
                df_xml[i].iloc[-1]=None

df_xml

,fecha,folio,formapago,lugarexpedicion,metodopago,moneda,nocertificado,serie,subtotal,tipocambio,...,version,conceptos,impuestos,emisor_nombre,emisor_regimenfiscal,emisor_rfc,receptor_nombre,receptor_rfc,receptor_usocfdi,xml
0,2020-01-19 20:56:42,30841,28.0,14090,PUE,MXN,1000000403853742,W,1046.08,1.0,...,3.3,"{'cfdi:Concepto': {'@Cantidad': '57.337', '@Cl...","{'@TotalImpuestosTrasladados': '192.42', 'cfdi...",UNIDAD LEO SA,601,ULE700330CD4,ALBERTO IBARRA DOMINGUEZ,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/31E406E0-3B33-...
1,2020-01-02 06:17:39,50052803708,3.0,14060,PUE,MXN,1000000404275186,DIC68694,0.01,NaN,...,3.3,None,None,BANCO INBURSA S.A. INSTITUCION DE BANCA MULTIP...,601,BII931004P61,ALBERTO IBARRA DOMINGUEZ,IADA810218HG5,P01,<_io.TextIOWrapper name='./temp/E10D677F-2027-...
2,2020-01-02 11:57:55,152723,1.0,14030,PUE,MXN,1000000502003151,BA,701.28,1.0,...,3.3,None,None,SUPER SERVICIOS SAN JUAN SA DE CV,601,SSS981105E38,ALBERTO IBARRA DOMINGUEZ,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/AF9469EC-9D07-...
3,2020-01-01 09:26:22,302014848195I,3.0,6820,PUE,MXN,1000000406542607,A,0.01,1.0,...,3.3,None,None,"SCOTIABANK INVERLAT, S. A.",601,SIN9412025I4,IBARRA DOMINGUEZ ALBERTO,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/0C0F5A44-83F9-...
4,2020-01-22 13:45:47,443096272,4.0,6600,PUE,MXN,1000000406929143,CFDB,241.09,1.0,...,3.3,None,None,"ESTAFETA MEXICANA, S.A. DE C.V.",601,EME880309SK5,ALBERTO IBARRA DOMINGUEZ,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/E240E30C-D381-...
5,2020-01-13 09:33:17,175,4.0,3100,PUE,MXN,1000000409894130,None,69827.59,NaN,...,3.3,None,None,IRONHACK MEXICO SC,601,IME171031RT6,Alberto Ibarra Domínguez,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/C85671F0-946F-...
6,2020-01-31 23:59:59,113332,5.0,10200,PUE,MXN,1000000500697732,B,15.52,NaN,...,3.3,None,None,MONEDEROS ELECTRONICOS ZION SA DE CV,601,MEZ141205FE7,ALBERTO IBARRA DOMINGUEZ,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/9D8DCB42-31AF-...
7,2020-01-05 11:43:18,1402819086,NaN,72700,None,XXX,1000000501452099,None,0.00,NaN,...,3.3,None,None,VOLKSWAGEN LEASING SA DE CV,601,VLE060918B86,ALBERTO IBARRA DOMINGUEZ,IADA810218HG5,P01,<_io.TextIOWrapper name='./temp/2B6FB11D-B8AB-...
8,2020-01-23 12:51:29,None,4.0,3100,PUE,MXN,1000000409894130,None,7758.62,NaN,...,3.3,None,None,IRONHACK MEXICO SC,601,IME171031RT6,Alberto Ibarra Domínguez,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/645F7212-5F0D-...
9,2020-01-08 13:16:10,2755,4.0,11270,PUE,MXN,1000000413791181,2F59C,39.84,NaN,...,3.3,None,None,FELIPE JONATHAN MORA IBARRA,621,MOIF750914662,ALBERTO IBARRA DOMINGUEZ,IADA810218HG5,G03,<_io.TextIOWrapper name='./temp/7268F561-12C4-...


In [ ]:
#clean_folder

#Descripción General: Funcion para borrar archivos de la carpeta temporal
#Recibe: ruta (path) por default una carpeta temp dentro de la ruta actual
#Recibe: lista de archivos (fl)
#Regresa: None

def clean_folder(path='./temp/',fl):
    for i in fl:
        remove(path+i)
    
    return None

In [283]:
def predict_encoding(file_path, n_lines=20):
    '''Predict a file's encoding using chardet'''
    import chardet

    # Open the file as binary data
    with open(file_path, 'rb') as f:
        # Join binary lines for specified number of lines
        rawdata = b''.join([f.readline() for _ in range(n_lines)])

    return chardet.detect(rawdata)['encoding']

In [ ]:
#Borrar archivos de temp
fileslist = [f for f in listdir(filespath) if isfile(join(filespath, f))]
clean_folder(fl=fileslist)